# 移す価値無

In [1]:
import cv2
import torch
import numpy as np
import time
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet101

# Initialize the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained model
model = deeplabv3_resnet101(pretrained=True).eval().to(device)

# Initialize camera
cap = cv2.VideoCapture(0)

# Wait for 5 seconds to capture the background
print("Capturing background in 5 seconds...")
time.sleep(5)
ret, background_frame = cap.read()

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()

    # Prepare the frame for the model
    input_frame = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])(frame).unsqueeze(0).to(device)

    # Make predictions
    with torch.no_grad():
        output = model(input_frame)['out'][0]
    output_predictions = torch.argmax(output, dim=0).byte().cpu().numpy()

    # Resize the output to match the frame size
    output_predictions_resized = cv2.resize(output_predictions, (frame.shape[1], frame.shape[0]))

# Replace the human part with the background
    human_mask = output_predictions_resized == 15
    replaced_frame = np.where(human_mask[:,:,None], background_frame, frame)


    # Show the frame
    cv2.imshow('Frame', replaced_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()


C:\Users\Manyan\anaconda3\envs\osac\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\Manyan\anaconda3\envs\osac\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Capturing background in 5 seconds...
Press 'q' to quit.
